# Detection of Pneumonia from Chest X-Ray Images (v1.0.0.3)
This notebook implements a CNN model to detect pneumonia from chest X-ray images using TensorFlow/Keras.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
from tqdm import tqdm
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

DATA_DIR = "/kaggle/input/chest-xray-pneumonia/chest_xray"
CATEGORIES = ["PNEUMONIA", "NORMAL"]
IMG_SIZE = 150

def load_images_from_folder(folder, label):
    data = []
    for filename in tqdm(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        try:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append([img, label])
        except:
            pass
    return data

train_data, val_data, test_data = [], [], []
for category in CATEGORIES:
    train_data += load_images_from_folder(os.path.join(DATA_DIR, "train", category), category)
    val_data += load_images_from_folder(os.path.join(DATA_DIR, "val", category), category)
    test_data += load_images_from_folder(os.path.join(DATA_DIR, "test", category), category)

random.shuffle(train_data)

def split_data(data):
    X = np.array([i[0] for i in data]) / 255.0
    y = np.array([i[1] for i in data])
    return X, y

X_train, y_train = split_data(train_data)
X_val, y_val = split_data(val_data)
X_test, y_test = split_data(test_data)

le = LabelEncoder()
y_train_enc = to_categorical(le.fit_transform(y_train), 2)
y_val_enc = to_categorical(le.transform(y_val), 2)
y_test_enc = to_categorical(le.transform(y_test), 2)